In [1]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class EfficientNetBackbone:
    """Pretrained EfficientNet backbone adapted for grayscale images"""
    def __init__(self, model_name='EfficientNetB0', input_shape=(512, 512, 1), freeze=True):
        self.model_name = model_name
        self.input_shape = input_shape
        self.freeze = freeze
        self.model = self.build_backbone()
        
    def build_backbone(self):
        """Create backbone with channel adaptation for grayscale"""
        # Input layer for grayscale images
        inputs = layers.Input(shape=self.input_shape)
        
        # Convert 1-channel to 3-channel by replication
        x = layers.Concatenate(axis=-1)([inputs, inputs, inputs])
        
        # Load pretrained EfficientNet
        if self.model_name == 'EfficientNetB0':
            base_model = tf.keras.applications.EfficientNetB0(
                include_top=False, 
                weights='imagenet',
                input_tensor=x
            )
            skip_layers = ['block2a_expand_activation', 'block3a_expand_activation']
        elif self.model_name == 'EfficientNetB4':
            base_model = tf.keras.applications.EfficientNetB4(
                include_top=False, 
                weights='imagenet',
                input_tensor=x
            )
            skip_layers = ['block2c_add', 'block4a_expand_activation']
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")
        
        # Freeze backbone if requested
        if self.freeze:
            base_model.trainable = False
            
        # Get skip connections
        skip_outputs = [base_model.get_layer(name).output for name in skip_layers]
        
        return Model(inputs=inputs, outputs=[base_model.output] + skip_outputs, name=f"{self.model_name}_Backbone")

class DeepLabV3Plus:
    """DeepLabV3+ decoder head for segmentation"""
    def __init__(self, num_classes):
        self.num_classes = num_classes
        
    def build_decoder(self, inputs, skip_connections):
        """Build DeepLabV3+ decoder"""
        # Unpack inputs
        x = inputs[0]  # Main backbone output
        skip1 = inputs[1]  # Low-level features (stride 4)
        skip2 = inputs[2]  # Mid-level features (stride 8)
        
        # ASPP Module
        x = self.aspp_module(x)
        
        # Decoder Path
        x = self.decoder_path(x, skip1, skip2)
        
        # Output layer
        return layers.Conv2D(self.num_classes, 1, activation='softmax', name='output')(x)
    
    def aspp_module(self, inputs):
        """Atrous Spatial Pyramid Pooling module"""
        # Get input shape
        input_shape = tf.shape(inputs)
        h, w = input_shape[1], input_shape[2]
        
        # Branch 1: 1x1 Conv
        b1 = layers.Conv2D(256, 1, padding='same', use_bias=False)(inputs)
        b1 = layers.BatchNormalization()(b1)
        b1 = layers.ReLU()(b1)
        
        # Branch 2: 3x3 Conv rate=6
        b2 = layers.Conv2D(256, 3, padding='same', dilation_rate=6, use_bias=False)(inputs)
        b2 = layers.BatchNormalization()(b2)
        b2 = layers.ReLU()(b2)
        
        # Branch 3: 3x3 Conv rate=12
        b3 = layers.Conv2D(256, 3, padding='same', dilation_rate=12, use_bias=False)(inputs)
        b3 = layers.BatchNormalization()(b3)
        b3 = layers.ReLU()(b3)
        
        # Branch 4: 3x3 Conv rate=18
        b4 = layers.Conv2D(256, 3, padding='same', dilation_rate=18, use_bias=False)(inputs)
        b4 = layers.BatchNormalization()(b4)
        b4 = layers.ReLU()(b4)
        
        # Branch 5: Image Pooling
        b5 = layers.GlobalAveragePooling2D()(inputs)
        b5 = layers.Reshape((1, 1, -1))(b5)
        b5 = layers.Conv2D(256, 1, use_bias=False)(b5)
        b5 = layers.BatchNormalization()(b5)
        b5 = layers.ReLU()(b5)
        b5 = layers.UpSampling2D(size=(h, w), interpolation='bilinear')(b5)
        
        # Concatenate and project
        x = layers.Concatenate()([b1, b2, b3, b4, b5])
        x = layers.Conv2D(256, 1, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        return layers.ReLU(name='aspp_output')(x)
    
    def decoder_path(self, inputs, skip1, skip2):
        """Upsampling path with skip connections"""
        # First upsample + skip connection
        x = layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(inputs)
        
        # Process low-level features
        skip1 = layers.Conv2D(48, 1, padding='same', activation='relu')(skip1)
        x = layers.Concatenate()([x, skip1])
        
        # Convolutional blocks
        x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
        x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
        
        # Second upsample + skip connection
        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        skip2 = layers.Conv2D(32, 1, padding='same', activation='relu')(skip2)
        x = layers.Concatenate()([x, skip2])
        
        # Final convolutions
        x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
        return layers.Conv2D(128, 3, padding='same', activation='relu', name='decoder_output')(x)

class SegmentationModel:
    """Complete segmentation model with EfficientNet backbone and DeepLabV3+ head"""
    def __init__(self, backbone_name='EfficientNetB0', num_classes=3, input_shape=(512, 512, 1)):
        self.backbone_name = backbone_name
        self.num_classes = num_classes
        self.input_shape = input_shape
        self.model = self.build_model()
        
    def build_model(self):
        """Assemble the complete model"""
        # Build backbone
        backbone = EfficientNetBackbone(
            model_name=self.backbone_name,
            input_shape=self.input_shape,
            freeze=True
        ).model
        
        # Get backbone outputs
        backbone_outputs = backbone.outputs
        backbone_input = backbone.input
        
        # Build decoder
        decoder = DeepLabV3Plus(num_classes=self.num_classes)
        outputs = decoder.build_decoder(backbone_outputs, backbone_outputs[1:])
        
        return Model(inputs=backbone_input, outputs=outputs, name='DeepLabV3Plus')
    
    def compile(self, learning_rate=1e-4):
        """Compile the model with standard settings"""
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
    
    def summary(self):
        return self.model.summary()
    
    def unfreeze_backbone(self, unfreeze_layers=10):
        """Selectively unfreeze backbone layers for fine-tuning"""
        # Unfreeze batch normalization layers first
        for layer in self.model.layers:
            if isinstance(layer, layers.BatchNormalization):
                layer.trainable = True
                
        # Unfreeze last N layers of backbone
        backbone = self.model.get_layer(f"{self.backbone_name}_Backbone")
        for layer in backbone.layers[-unfreeze_layers:]:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Initialize and compile model
model = SegmentationModel(
    backbone_name='EfficientNetB0',
    num_classes=3,  # Background + your classes
    input_shape=(512, 512, 1)
)

# See model architecture
model.summary()

# Compile with custom learning rate
model.compile(learning_rate=1e-4)

# Prepare dataset (example using tf.data)
def load_data(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, (512, 512))
    
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, (512, 512), method='nearest')
    
    return image, mask

# Create dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_image_paths, train_mask_paths))
train_ds = train_ds.map(load_data).batch(4)

# Train initial model
model.model.fit(train_ds, epochs=10)

# Fine-tune with backbone unfrozen
model.unfreeze_backbone(unfreeze_layers=15)
model.compile(learning_rate=1e-5)
model.model.fit(train_ds, epochs=5)